In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     #get fit parameters for scaling
X_valid = scaler.transform(X_valid)         # scale validation and test test using those parameters
X_test = scaler.transform(X_test)

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation = 'relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer = "sgd")

In [7]:
history = model.fit(X_train, y_train,
                    epochs = 20,
                    validation_data=(X_valid,y_valid)
)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.0050 - val_loss: 0.5697
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5131 - val_loss: 0.4763
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4781 - val_loss: 0.4583
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4657 - val_loss: 0.4500
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4540 - val_loss: 0.4432
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4475 - val_loss: 0.4539
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4422 - val_loss: 0.4325
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4321 - val_loss: 0.4317
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4287 - val_loss: 0.4250
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4207 - val_loss: 0.4181

In [8]:
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.3835


In [9]:
X_new = X_test[:3]  #pretend that these are new instances
y_pred = model.predict(X_new)

1/1 [==============================] - 0s 50ms/step


### Keras's Functional API

In [10]:
# Wide and Deep NN
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])


In [11]:
# A variation: sending subset of features through the wide path and a different subset through the deep path

input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])       # 5+30 = 35
output = keras.layers.Dense(1, name='output')(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])


In [12]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))


c:\Users\viraj\miniconda3\envs\tfGPU\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:,2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [14]:
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data = ((X_valid_A, X_valid_B), y_valid)
                    )

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.8685 - val_loss: 0.7486
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6864 - val_loss: 0.5939
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5864 - val_loss: 0.5448
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5485 - val_loss: 0.5209
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5269 - val_loss: 0.5049
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5124 - val_loss: 0.4935
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5012 - val_loss: 0.4843
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4932 - val_loss: 0.4780
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4872 - val_loss: 0.4734
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4823 - val_loss: 0.4696

In [15]:
mse_test = model.evaluate((X_test_A, X_test_B), y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.4566


In [16]:
y_pred = model.predict((X_new_A, X_new_B))

1/1 [==============================] - 0s 26ms/step


In [17]:
# adding an auxillary output for regularization

output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output,aux_output])



In [18]:
# each output will need its own loss function. Moreover, aux_output is only for regularization we apply much less weight to it
model.compile(loss=['mse', 'mse'], loss_weights=[0.9,0.1], optimizer="sgd")


In [19]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=20,
    validation_data = ([X_valid_A, X_valid_B], [y_valid, y_valid])
)

Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 0.8346 - main_output_loss: 0.7163 - aux_output_loss: 1.8993 - val_loss: 0.5239 - val_main_output_loss: 0.4701 - val_aux_output_loss: 1.0081
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5449 - main_output_loss: 0.4986 - aux_output_loss: 0.9620 - val_loss: 0.5004 - val_main_output_loss: 0.4591 - val_aux_output_loss: 0.8721
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5510 - main_output_loss: 0.5155 - aux_output_loss: 0.8714 - val_loss: 0.4839 - val_main_output_loss: 0.4514 - val_aux_output_loss: 0.7762
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5122 - main_output_loss: 0.4815 - aux_output_loss: 0.7894 - val_loss: 0.4807 - val_main_output_loss: 0.4533 - val_aux_output_loss: 0.7275
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4890 - main_output_loss: 0.4623 - aux_output_loss: 0.7297 - val_loss

In [20]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test,y_test]
)

162/162 [==============================] - 0s 2ms/step - loss: 0.3695 - main_output_loss: 0.3513 - aux_output_loss: 0.5334


In [21]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 32ms/step


# As it can be noticed, you can build any sort of architecture you want quite easily with the Functional API

# Saving and Restoring a Model

In [22]:
model.save("worked_example_model.h5")